# Lesson 6.1: Prompt Engineering

**Note:** In this lesson you will be given examples for creating prompts based on using data and pandas as well as non-data examples. You will be shown how to structure your prompts for both types of examples.

## <ins>Setup</ins>  

**Reminder:** Don't forget to setup your virtual environment, choose your python interpreter and install the ipykernel.

### 1. Retrieving Your API Key

Before we begin, you will need to retrieve your API key from Gemini.

Use the following set of instructions to sign up for an account and retrieve your API key.

[Gemini API Key](https://github.com/jdrichards-pursuit/gemini-api-key-acquire?tab=readme-ov-file)

### 2. Setting Up Your Environment Variables

Now that you have your API key, you can set up your environment variables.

Create a new file called `.env` and add the following line of code:

```bash
API_KEY=<your-api-key>
```

### 3. Installing Required Libraries

Now that you have your API key, you can install the required libraries.

```bash
pip install python-dotenv
```

### 4. Importing Required Libraries

Now that you have your API key, you can import the required libraries. When you run this code, it will print your API key to the console.




In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("API_KEY")

print(api_key)

After viewing your API key, you may now want to remove the print statement for security reasons.

### 5. Importing Required Libraries

For this lesson you will also need to install the following libraries:

```bash
pip install google-generativeai pandas
```

### 6. Import Packages

Next, import these packages into your environment.


In [ ]:
import google.generativeai as genai
import pandas as pd

### 7. Set Up API Key

You will need to set up your API key using your already created environment variable.


In [ ]:
os.environ['API_KEY'] = api_key
genai.configure(api_key=os.environ['API_KEY'])

## <ins>Prompt Process Under the Hood</ins>

Before we begin digging into prompt engineering, let's familiarize ourselves on a high level with what happens under the hood when we send a prompt. This is definitely 'above our paygrade' and not what we will be focusing on, but it is good to be aware of. Take a look at this link which describes [The Process of A Prompt](https://github.com/jdrichards-pursuit/prompt-process-explained).

### Parts of a Prompt

When creating a prompt using the Gemini API, there are three main parts to consider:

1. **Model**: The model you are using to generate the response. In our case, we are using the `gemini-1.5-flash` model.
2. **Parameters**: Parameters are additional settings or options that you can send to the model to guide the response. We will cover these in more detail later in the lesson.
3. **Prompt**: A prompt is the input or instruction given to an AI model to elicit a specific response or output. It's the text that guides the AI in understanding what kind of information or task you're requesting.


### Model
For this lesson, we will be using the `gemini-1.5-flash` model because it is free and '...is a fast and versatile multimodal model for scaling across diverse tasks.' to quote the [Gemini API Documentation](https://ai.google.dev/gemini-api/docs/models/gemini#gemini-1.5-flash).




### Parameters

When we send a prompt to the model, we can also send parameters to the model. These parameters are used to guide the model's response. In Gemini API, there are several parameters we can send to the model, but for now we will focus on the following:

- `temperature`: This parameter controls the randomness of the model's response. A temperature of 0 is deterministic, meaning the model will always return the same response for a given prompt. A temperature between 0 and 1 will make the response more random.
- `max_output_tokens`: This parameter controls the maximum number of tokens in the model's response.
- `num_return_sequences`: This parameter controls the number of alternative completions to generate.
- `min_length`: This parameter controls the minimum number of tokens in the model's response.
- `max_length`: This parameter controls the maximum number of tokens in the model's response.


There is also a are many more parameters, but we will not focus on those for now.

#### 1. Temperature

- This parameter controls the randomness of the generated text.

- Think of it like adjusting the "creativity" or "originality" of the generated text.

- A higher temperature (e.g., 0.9) means the API will generate more unique and creative text, but it might also be less coherent or grammatically correct.

- A lower temperature (e.g., 0.1) means the API will generate more predictable and coherent text, but it might also be less creative or original.
By default, the API uses a temperature of 0.5, which is a good balance between creativity and coherence.

- By default, the API uses a temperature of 0.5, which is a good balance between creativity and coherence.

#### 2. Max Output Tokens

- This parameter sets the maximum number of tokens in the generated text.
- A token is the smallest unit of text that the model can generate.
- If you set max_output_tokens to 20, the API will generate text that's around 20 tokens long.

#### 3. Num Return Sequences

- This parameter sets the number of alternative completions to generate.
- If you set num_return_sequences to 3, the API will generate 3 different completions for the same prompt.


#### 4. Min Length

- This parameter sets the minimum number of characters (letters, spaces, punctuation) in the generated text.
- Think of it like setting a word count for a minimum number of sentences. You might ask someone to write at least 5 sentences.
- The API will generate text that's as close to the minimum length as possible, but it might not reach the exact limit.
- If you set min_length to 20, the API will generate text that's around 20 characters long.
- There is no default value for this parameter, so you will always need to set it.

#### 5. Max Length

- This parameter sets the maximum number of characters (letters, spaces, punctuation) in the generated text.
- Think of it like setting a word count for a maximum number of sentences. You might ask someone to write at most 5 sentences.
- The API will generate text that's as close to the maximum length as possible, but it might not reach the exact limit.
- If you set max_length to 20, the API will generate text that's around 20 characters long.
- There is no default value for this parameter, so you will always need to set it.

### Default Parameters

You do not have to set all of these parameters and we will not explain them all,, but it is good to be aware of them. When they are not specified, the API will use the default values. The defaults are:

```python
generation_config = {
    "model": "gemini-1.5-flash",
    "temperature": 0.7,
    "max_output_tokens": 2048,
    "num_return_sequences": 1,
    "top_p": 1,
    "top_k": 32,
    "stop_sequences": [],
    "candidate_count": 1,
    "safety_settings": [
        {
            "category": "HARM_CATEGORY_HARASSMENT",
            "threshold": "BLOCK_MEDIUM_AND_ABOVE"
        },
        {
            "category": "HARM_CATEGORY_HATE_SPEECH",
            "threshold": "BLOCK_MEDIUM_AND_ABOVE"
        },
        {
            "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
            "threshold": "BLOCK_MEDIUM_AND_ABOVE"
        },
        {
            "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
            "threshold": "BLOCK_MEDIUM_AND_ABOVE"
        }
    ],
    "prompt_feedback": {
        "safety_ratings": []
    }
}
```

### Lets create a function, `get_completion`, that will submit a prompt to the Gemini API and return the response:

In [ ]:
def get_completion(prompt, model="gemini-1.5-flash", **kwargs):
    model = genai.GenerativeModel(model)
    
    # Create a generation_config dictionary with default values
    generation_config = {
        "temperature": 0.8,
        "min_length": 20,
        "max_length": 200,
    }
    
    # Update generation_config with any provided kwargs
    generation_config.update(kwargs)
    
    response = model.generate_content(prompt, generation_config=generation_config)
    return response.text

#### Walkthrough of the Code

1. We first create a function called `get_completion` that takes in a prompt and any number of keyword arguments called `kwargs`.
    - `**kwargs` allows us to pass in any number of keyword arguments to the function.
    - the double asterisk is used to unpack the keyword arguments into a dictionary. In JavaScript, this would be `{...args}`.

2. We then create a `generation_config` dictionary with default values for the parameters we want to default to.

3. We then update the `generation_config` dictionary with any provided keyword arguments. This allows us to override the default values when we call the function.

4. We then generate the response from the model using the `generate_content` method on the model dictionary.

5. Finally, we return the response.

### Let's call the function with our prompt and print the response:

In [ ]:
response = get_completion(
    prompt,
    temperature=0.9,
    max_output_tokens=1000
)

print(response)

## <ins>Prompt Engineering Techniques</ins>

Prompt engineering is a crucial skill for effectively leveraging AI language models like ChatGPT. It involves crafting inputs that guide the AI to produce desired outputs. Mastering these techniques allows users to get more accurate, relevant, and useful responses from AI systems.

Key prompt engineering techniques include:
- Formulating clear and specific questions
- Providing relevant context
- Specifying desired output format
- Including examples to anchor prompts
- Breaking down complex tasks into steps
- Encouraging multiple alternatives
- Iterative prompting and refinement

These techniques help improve the AI's reasoning capabilities, enable more structured outputs, and enhance overall interaction quality with language models.

### 1. Formulating Effective Questions

Whether you're analyzing data or seeking general information, asking clear and specific questions is crucial. Let's explore how to frame your queries effectively in both data-related and general contexts.

#### Clear vs. Vague Instructions

#### Data Example:
```python
# Good: Clear and specific question
"""
What is the average sale price of houses with more than 3 bedrooms 
in the 'Seattle' area from our housing dataset?
"""

# Bad: Vague and ambiguous question
"""
What can you tell me about the houses in the dataset?
"""
```

#### Non-Data Example:
```python
# Good: Clear and specific question
"""
What are the three main causes of deforestation in the Amazon rainforest 
over the past decade?
"""

# Bad: Vague and ambiguous question
"""
Tell me about forests.
"""
```

### Practice
Imagine you're asking someone to explain the concept of climate change.

1. Write a vague instruction for explaining climate change.
2. Then, rewrite the instruction to make it clear and specific.
3. Show the class what you wrote for both prompts and explain the differences in the two model responses.

In [ ]:
# comment and uncomment your prompts one at a time to see the difference
# Your vague instruction:
prompt = """
"""
# Your clear and specific instruction:

prompt = """
"""

response = get_completion(
    prompt
)

print(response)

### 2. Providing Relevant Context

##### Data Example:
```python
# Good: Includes relevant context
"""
Our dataset 'sales.csv' contains columns for 'price', 'bedrooms', and 'location'.
Can you show me how to calculate the median price for each bedroom category?
"""

# Bad: Lacks necessary context
"""
Calculate the median prices.
"""
```

##### Non-Data Example:
```python
# Good: Includes relevant context
"""
In the context of object-oriented programming in Python, 
can you explain the concept of inheritance with a simple example?
"""

# Bad: Lacks necessary context
"""
What is inheritance?
"""
```

### Practice
Imagine you're asking for advice on how to care for a specific type of houseplant.

1. Write a prompt without providing any context.
2. Then, rewrite the prompt to include relevant context.
3. Show the class what you wrote for both prompts and explain the differences in the two model responses.

Relevant context for houseplant care could include:
- Plant species or variety
- Plant size and age
- Light conditions
- Location in the home
- Temperature and humidity levels
- Current plant health
- Watering habits


In [ ]:
# comment and uncomment your prompts one at a time to see the difference
# Your prompt without context:
prompt = """
"""
# Your prompt with context:
prompt = """
"""

response = get_completion(
    prompt
)

print(response) # this is the response from the model


### 3. Specifying Constraints in Your Queries

When working with data or seeking information, it's important to specify any constraints or limitations in your request. This helps ensure you get precisely the information you need in the format you want.

#### Data Example:

##### Good: Specifies constraints clearly

```python
"""
From our 'employee_data.csv', analyze the salary distribution for employees 
who joined in the last 2 years. Provide the 25th, 50th, and 75th percentiles, 
and limit your response to a maximum of 4 bullet points.
"""
```

##### Bad: Lacks specific constraints

```python
"""
Analyze the salary distribution from the employee data.
"""
```

#### Non-Data Example:

##### Good: Specifies constraints clearly

```python
"""
Explain the concept of recursion in programming. Use exactly 3 sentences 
and provide a simple example that doesn't involve calculating factorials 
or Fibonacci numbers.
"""
```

##### Bad: Lacks specific constraints

```python
"""
Explain recursion in programming.
"""
```

Remember: By specifying constraints such as time periods, specific metrics, response format, or content limitations, you can ensure that the response meets your needs exactly. This is crucial whether you're analyzing data from a CSV file or seeking general information on a topic.

### Practice

Imagine you're asking for recommendations for a book to read.

1. Write a prompt without specifying any constraints.
2. Then, rewrite the prompt to include specific constraints. 
3. Show the class what you wrote for both prompts and explain the differences in the two model responses.

Consider including constraints such as:

- Genre (e.g., science fiction, mystery, historical fiction)
- Publication date (e.g., published in the last 5 years)
- Awards or recognition (e.g., has won at least one major literary award)
- Themes or topics (e.g., features themes of artificial intelligence or space exploration)
- Target audience (e.g., suitable for young adult readers)
- Length (e.g., under 400 pages long)
- Format of the recommendation (e.g., provide title, author, and a brief description)


### 4. Specifying Output Format

When working with data or seeking information, you can guide the model to provide responses in a specific format. This is particularly useful when you need structured outputs for further processing or analysis. This structured output can be in the form of a JSON, a table, a list, etc. Your example for this structure can either be a partial example or a full example.

##### Data Example:

#### Prompt without format specification:

```python
"""
Analyze the top 3 selling products from our 'annual_sales.csv' file.
"""
```

#### Possible Response:

```
Based on the analysis of the 'annual_sales.csv' file, the top 3 selling products are:

1. Smartphone X: This product led in units sold with 50,000 units, generating a revenue of $25,000,000.

2. Laptop Pro: While selling fewer units (30,000) than the Smartphone X, it generated the highest revenue at $45,000,000 due to its higher price point.

3. Wireless Earbuds: This product showed strong unit sales at 100,000, but had a lower revenue of $15,000,000, indicating a lower price point.

These top products demonstrate a mix of high-volume and high-value items, with smartphones leading in units sold and laptops in revenue generation.
```

To ensure a specific JSON format, you can provide the beginning of the JSON structure:

#### Prompt with format specification (using partial JSON structure):

```python
prompt = """
Analyze the top 3 selling products from our 'annual_sales.csv' file. 
Provide the output in the following JSON format:

{
  "top_products": [
    {
      "rank": 1,
      "name": 
"""
```

#### Possible Response:

```json
{
  "top_products": [
    {
      "rank": 1,
      "name": "Smartphone X",
      "units_sold": 50000,
      "revenue": 25000000
    },
    {
      "rank": 2,
      "name": "Laptop Pro",
      "units_sold": 30000,
      "revenue": 45000000
    },
    {
      "rank": 3,
      "name": "Wireless Earbuds",
      "units_sold": 100000,
      "revenue": 15000000
    }
  ],
  "analysis": "The top 3 products demonstrate a balance between high-volume and high-value items. Smartphone X leads in units sold, while Laptop Pro generates the highest revenue. Wireless Earbuds show strong unit sales but lower per-unit revenue."
}
```

## Non-Data Example:

#### Prompt without format specification:

```python
prompt = """
Describe the key features of Python programming language.
"""
```

#### Possible Response:

```
Python is a popular programming language known for several key features:

1. Easy to learn and read syntax: Python uses indentation and simple keywords, making it accessible for beginners and easy to read for all developers.

2. Interpreted language: Python code is executed line by line, allowing for rapid development and debugging.

3. Dynamically typed: Variables in Python can change types, providing flexibility in programming.

4. Object-oriented programming support: Python supports OOP concepts like classes, inheritance, and encapsulation.

5. Extensive standard library: Python comes with a large library of pre-built modules, reducing the need for external dependencies.

6. Cross-platform compatibility: Python programs can run on various operating systems with minimal modifications.

Python is widely used in web development, data analysis, artificial intelligence, machine learning, scientific computing, and automation tasks. The latest stable version as of May 2021 is 3.9.5.
```

To ensure a specific JSON format, you can provide the beginning of the JSON structure:

#### Prompt with format specification (using partial JSON structure):

```python
prompt = """
Describe the key features of Python programming language. 
Provide the output in the following JSON format:

{
  "language": "Python",
  "features": [
    {
      "name": 
"""
```

#### Possible Response:

```json
{
  "language": "Python",
  "features": [
    {
      "name": "Easy to learn and read syntax",
      "description": "Python uses indentation and simple keywords, making it accessible for beginners and easy to read for all developers."
    },
    {
      "name": "Interpreted language",
      "description": "Python code is executed line by line, allowing for rapid development and debugging."
    },
    {
      "name": "Dynamically typed",
      "description": "Variables in Python can change types, providing flexibility in programming."
    },
    {
      "name": "Extensive standard library",
      "description": "Python comes with a large library of pre-built modules, reducing the need for external dependencies."
    },
    {
      "name": "Cross-platform compatibility",
      "description": "Python programs can run on various operating systems with minimal modifications."
    }
  ],
  "version": "3.9.5",
  "popular_applications": ["Web development", "Data analysis", "AI and machine learning", "Scientific computing"]
}
```

Remember: By providing a structure for the desired output format, you can guide the model to generate responses in a consistent and easily parseable format, whether you're analyzing data from a CSV file or seeking information on programming concepts.

### Practice

Imagine you're asking for information about a historical event.

1. Write a prompt without specifying any output format.
2. Rewrite the prompt to include a specific output format.
3. Show the class what you wrote for both prompts and explain the differences in the two model responses.

**Historical events could include:**

- World War II
- The Great Fire of London
- The French Revolution
- The American Revolution  

**Consider formats such as:**

- Bulleted list
- Timeline
- Table
- Pros and cons list
- Q&A format
- Step-by-step guide
- Comparison chart
- Brief paragraph summary


In [ ]:
# comment and uncomment your prompts one at a time to see the difference
# Your prompt without specifying output format:
prompt = """
"""
# Your prompt with specifying output format:
prompt = """
"""

response = get_completion(
    prompt
)

print(response) # this is the response from the model

### 5. Including Few-Shot Examples in Your Queries

When working with data or seeking information, providing examples in your prompt can help guide the model to produce more accurate and appropriately formatted responses. This technique is called "few-shot learning" and can be applied to both data analysis tasks and general inquiries.

#### Definitions:

- **Zero-shot prompt**: A prompt that provides no examples, relying solely on instructions.
- **One-shot prompt**: A prompt that includes a single example to guide the model's response.
- **Few-shot prompt**: A prompt that contains multiple examples to demonstrate the desired pattern or format.

#### Data Example:

##### Zero-shot prompt:

```python
prompt = """
Analyze the sales data from our 'quarterly_sales.csv' file and provide a summary 
of the top-performing products.
"""
```

##### Few-shot prompt:

```python
prompt = """
Here are two examples of how to analyze and summarize sales data:

Example 1:
Input: Analyze Q1 sales for electronic products
Output:
1. Top product: Smartphones (Revenue: $1.2M, Units sold: 5000)
2. Fastest growing: Wireless earbuds (300% growth from last quarter)
3. Underperformer: Tablets (20% decline in sales)

Example 2:
Input: Summarize Q2 sales for home appliances
Output:
1. Best seller: Robot vacuums (Revenue: $800K, Units sold: 2000)
2. Surprise hit: Smart thermostats (150% increase in demand)
3. Needs attention: Microwave ovens (10% drop in market share)

Now, using the same format, analyze the sales data from our 'quarterly_sales.csv' 
file and provide a summary of the top-performing products for Q3.
"""
```

#### Non-Data Example:

##### One-shot prompt:

```python
prompt = """
Here's an example of how to explain a programming concept concisely:

Concept: List comprehension in Python
Explanation: A compact way to create lists based on existing lists or iterables, 
combining a for loop and a conditional statement in a single line of code.

Now, using the same concise format, explain the concept of 'decorator' in Python.
"""
```

##### Few-shot prompt:

```python
prompt = """
Here are two examples of explaining programming concepts:

Concept: List comprehension in Python
Explanation: A compact way to create lists based on existing lists or iterables, 
combining a for loop and a conditional statement in a single line of code.

Concept: Lambda functions in Python
Explanation: Anonymous, inline functions defined using the 'lambda' keyword, 
typically used for simple operations and as arguments to higher-order functions.

Now, using the same concise format, explain the following two concepts:
1. Decorator in Python
2. Generator expression in Python
"""
```

Remember: Including examples in your prompts can significantly improve the relevance and format of the responses, whether you're analyzing data from a CSV file or seeking explanations for programming concepts.

### Practice

Your goal is to create prompts that will instruct a language model to convert informal text messages into more formal, professional language.

1.  Write a zero-shot prompt for this task without providing any examples.
2. Then, write a few-shot prompt that includes at least two examples to guide the model's response.
3. Show the class what you wrote for both prompts and explain the differences in the two model responses.

Suggested example type:

Informal: "Hey! Can u send me that report ASAP? Thx!"
Formal: "Hello, Could you please send me that report as soon as possible? Thank you."

Use your own examples to practice.


In [ ]:
# comment and uncomment your prompts one at a time to see the difference
# Your zero-shot prompt:
prompt = """
"""
# Your few-shot prompt:
prompt = """
"""

response = get_completion(
    prompt
)

print(response) # this is the response from the model   

### 6. System Instructions: Shaping AI Behavior

System instructions, also known as system prompts, are powerful tools for defining how an AI model should behave and respond. These instructions can be used to create specialized assistants, set response formats, and limit the scope of the AI's knowledge or capabilities.

By using system instructions, you can essentially create a custom `AI Assistant` tailored to specific tasks or domains. For example, you could create:

- A data analysis assistant
- A language tutor
- A creative writing coach
- A technical support specialist

The system instruction sets the "personality" and capabilities of the assistant.

Certainly! I'll provide both a good and a bad example of using system instructions to create a specialized assistant. This contrast will help illustrate the importance of well-crafted system instructions.

#### Bad Example: Vague General Assistant

##### System Instruction (Bad):

```python
system_instruction = """
You are a helpful assistant. Answer user questions.
"""
```

##### User Prompt:

```python
user_prompt = """
I'm a 30-year-old woman, 165 cm tall, weighing 70 kg. I exercise 3 times a week. 
Can you suggest a balanced meal plan for weight loss?
"""

full_prompt = f"{system_instruction}\n\n{user_prompt}"
response = get_completion(full_prompt)
print(response)
```

##### Possible Response:

```
Certainly! I'd be happy to suggest a balanced meal plan for weight loss. Here's a general outline:

Breakfast:
- Whole grain toast with avocado and eggs
- Greek yogurt with berries and nuts

Lunch:
- Grilled chicken salad with mixed greens and vinaigrette dressing
- Quinoa bowl with vegetables and lean protein

Dinner:
- Baked fish with roasted vegetables
- Stir-fry with tofu and brown rice

Snacks:
- Apple slices with peanut butter
- Carrot sticks with hummus

Remember to control portion sizes and stay hydrated. It's also important to consult with a nutritionist or doctor for personalized advice.

Good luck with your weight loss journey!
```

The bad example lacks specificity and fails to:
1. Ask for additional necessary information
2. Provide specific portion sizes or calorie counts
3. Explain the nutritional benefits of the suggestions
4. Format the response in an easily readable manner
5. Use consistent units of measurement
6. Offer a comprehensive plan tailored to the user's needs

#### Good Example: Nutrition Advisor Assistant

##### System Instruction (Good):

```python
system_instruction = """
You are a nutrition advisor specialized in providing dietary recommendations. 
When responding to queries:
1. Always ask for the user's age, weight, height, and activity level if not provided.
2. Provide advice based on scientifically backed nutritional guidelines.
3. Offer meal suggestions with specific portion sizes.
4. Include a brief explanation of the nutritional benefits for each recommendation.
5. If asked about medical conditions, remind the user to consult with a healthcare professional.
6. Format responses in easy-to-read bullet points or numbered lists.
7. Use metric units (kg, cm) with imperial conversions in parentheses.

Do not provide advice on topics unrelated to nutrition and healthy eating habits.
"""
```

##### User Prompt:

```python
user_prompt = """
I'm a 30-year-old woman, 165 cm tall, weighing 70 kg. I exercise 3 times a week. 
Can you suggest a balanced meal plan for weight loss?
"""

full_prompt = f"{system_instruction}\n\n{user_prompt}"
response = get_completion(full_prompt)
print(response)
```

##### Possible Response:

```
Certainly! Based on the information you've provided, here's a balanced meal plan to support your weight loss goals:

1. Breakfast (300-350 calories):
   • 1 cup (90g) of oatmeal with 1 tbsp (15g) of chia seeds
   • 1 medium apple, sliced
   • 200ml of unsweetened almond milk
   Benefits: High in fiber for satiety, provides slow-release energy

2. Mid-morning Snack (100-150 calories):
   • 1 small handful (30g) of unsalted mixed nuts
   Benefits: Healthy fats and protein to curb hunger

3. Lunch (400-450 calories):
   • 120g grilled chicken breast
   • 1 cup (150g) of mixed salad greens
   • 1/2 cup (75g) of cherry tomatoes
   • 1/4 avocado, sliced
   • 1 tbsp (15ml) of olive oil and balsamic vinegar dressing
   Benefits: Lean protein for muscle maintenance, vegetables for vitamins and fiber

4. Afternoon Snack (100-150 calories):
   • 1 medium carrot, sliced
   • 2 tbsp (30g) of hummus
   Benefits: Crunchy texture satisfies cravings, protein and fiber for satiety

5. Dinner (400-450 calories):
   • 150g baked salmon
   • 1 cup (150g) of roasted Brussels sprouts
   • 1/2 cup (75g) of quinoa
   Benefits: Omega-3 fatty acids from salmon, complex carbs from quinoa for sustained energy

6. Evening Snack (if needed, 100 calories):
   • 1 cup (150g) of mixed berries
   Benefits: Low in calories, high in antioxidants and fiber

Remember to stay hydrated by drinking at least 8 glasses (2 liters) of water throughout the day. This meal plan provides approximately 1500-1600 calories, which should create a moderate calorie deficit for weight loss given your stats and activity level.

Always listen to your body and adjust portions if needed. For personalized medical advice or if you have any health conditions, please consult with a healthcare professional or registered dietitian.
```



This comparison demonstrates how well-crafted system instructions can lead to more detailed, helpful, and tailored responses, while vague instructions may result in generic advice that doesn't fully address the user's specific situation or needs.

By using system instructions, you can create AI assistants that are specialized in certain areas, adhere to specific formatting requirements, and have defined boundaries of knowledge or capabilities. This approach allows for more controlled and targeted interactions with AI models.

### Practice: Creating a Specialized AI Travel Assistant with System Instructions

1. Write a non-specific prompt without specifying any system instructions for a travel assistant.
2. Rewrite the prompt to include a specific system instruction for a travel assistant. 
3. Show the class what you wrote for both prompts and explain the difference in responses from the model.

Consider including instructions for (not necessarily all of these or limited to these):

- Asking for specific user information (e.g., destination, dates, budget, preferences)
-  Providing recommendations based on current travel information
- Offering detailed itinerary suggestions with costs and time estimates
- Including descriptions of attractions and activities
Reminding users about official sources for visa and health information
- Formatting responses for easy reading
- Handling currency conversions
- Suggesting money-saving tips
- Providing cultural information
- Setting boundaries on what services the assistant can and cannot provide

**Note:** You will want to create your system instruction first, then redefine your get_completion function to include the system instruction as part of the prompt. See below for an example.




In [ ]:
def get_completion(prompt,system_instruction=None, model="gemini-1.5-flash", **kwargs):
    model = genai.GenerativeModel(model)
    
    if system_instruction:
        prompt = f"{system_instruction}\n\n{prompt}"
    
    # Create a generation_config dictionary with default values
    generation_config = {
        "temperature": 0.7,
        "min_length": 20,
        "max_length": 200,
        "max_output_tokens": 2020,
    }
    
    # Update generation_config with any provided kwargs
    generation_config.update(kwargs)
    
    response = model.generate_content(prompt, generation_config=generation_config)
    return response.text

In [ ]:
# comment and uncomment your prompts one at a time to see the difference
# Your basic system instruction:
system_instruction = """
"""
# Your more effective and specialized system instruction:
system_instruction = """
"""                         

